In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, date, datetime
import folium
from folium.plugins import TimestampedGeoJson
import networkx as nx


class Patient:

    def __init__(self, first_name, last_name, health):
        self.first_name = first_name
        self.last_name = last_name
        self.health = health
        self.friends = []
        self.location = []

    def add_friend(self, friend_person):
        self.friends.append(friend_person)

    def add_location(self, location):
        self.location = location

    def get_friends(self):
        return self.friends
    
    def get_location(self):
        return self.location
    
    def get_name(self):
        return self.first_name + self.last_name

    def get_health(self):
        return self.health

    def infect(self, viral_load):
        if self.health <= 29:
            self.health -= 0.1 * viral_load

        elif 29 < self.health < 50:
            self.health -= viral_load

        elif self.health >= 50:
            self.health -= 2 * viral_load

        if self.health < 0:
            self.health = 0

    def is_contagious(self):
        if round(self.health, 0) < 50:
            return True
        else:
            return False

    def sleep(self):
        if self.health < 96:
            self.health += 5
        else:
            self.health = 100

    def set_health(self, new_health):
        self.health = new_health
    
    def set_location(self, location):
        self.location = location
    
    def __str__(self):
        return self.first_name + " " + self.last_name

    
def collect_data(start_date, end_date):
    for patient in patients:
        position = patient.get_location()
        coordinate.append(position)
        
    
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

        
def load_patients(initial_health):
    f = open("/Users/MichaelFu/Desktop/a2_sample_set222.txt", "r")
    f1 = f.readlines()
    patients = []
    
    for line in f1:
        line_split_list = line.split(":")
        patient_split_list = line_split_list[0].split()
        one_patient = Patient(patient_split_list[0], patient_split_list[1], initial_health)
        patients.append(one_patient)

    cur_index = 0
    for line in f1:
        line_split_list = line.split(":")
        all_patient_split_list = line_split_list[1].split(",")

        for patient_name in all_patient_split_list:
            single_patient_split_list = patient_name.split()
            name = single_patient_split_list[0].strip() + single_patient_split_list[1].strip()

            for patient in patients:
                if patient.get_name() == name:
                    patients[cur_index].add_friend(patient)
                    break
        cur_index += 1
    f.close()
    return patients

def locate_patients(patients):
    for patient in patients:
        south_boundary = -37.9656
        north_boundary = -37.8726
        east_boundary = 145.2330
        west_boundary = 145.0247
        location = [random.uniform(west_boundary, east_boundary), random.uniform(south_boundary, north_boundary)]
        patient.add_location(location)

        
# use edges to make meeting happen, this version wouldn't double the num of meetings
def run_simulation(start_date, end_date, meeting_probability, patient_zero_health):  
    # load patient from txt file with specific health point
    patients = load_patients(75)
    locate_patients(patients)
    # set first patient health
    first_patient = patients[0]
    first_patient.set_health(patient_zero_health)
    
    total_contagious = []
    original_node_color = []
    G = nx.Graph()    
    G.add_nodes_from(patients)
    
    # set edges after setting up the nodes, and the node color
    for man in patients:
        if man.is_contagious():
                original_node_color.append('red')
        else:
            original_node_color.append('green')
        
        friends = man.get_friends()       
        for friend in friends:
            if G.has_edge(man, friend) == False:
                G.add_edge(man, friend, color='gray', weight=1)
    
    # get all of the edges
    edges = G.edges()
    # get list of color and weight from all of the edges
    colors = [G[man][friend]['color'] for man,friend in edges]
    weights = [G[man][friend]['weight'] for man,friend in edges]
    
    # after setting nodes and edges, specify the layout
    nodes_pos = nx.spring_layout(G)

    plt.figure(3,figsize=(25, 25))         
    nx.draw_networkx(G, pos=nodes_pos, node_size=120, node_color=original_node_color, 
                     edge_color=colors, width=weights)        
    plt.title('day0', fontsize=40)
    plt.savefig('originalNetwork.png')
    plt.show()


    day = 1
    
    
    
    for single_date in daterange(start_date, end_date):
        day_contagious = 0
        # specification of node color and edge color
        node_color = []
        edge_color = []
        edge_width = []
        date = single_date.strftime('%Y-%m-%d')
        print(date)
        index_color = 0
        index_weight = 0
        for man, friend in edges:
            viral_load = 5 + ((man.get_health() - 25) ** 2) / 62
            viral_load_friend = 5 + ((friend.get_health() - 25) ** 2) / 62
            r = random.randint(1, 100) / 100                
            # black edges mean meeting does not happen    
            if meeting_probability < r:
                colors[index_color] = 'black'
                weights[index_weight] = 1
            # purple lines mean single or dual direction contagious meeting happens
            # dual contagious meeting happens
            elif meeting_probability > r and man.is_contagious() and friend.is_contagious():
                man.infect(viral_load_friend)
                friend.infect(viral_load)
                colors[index_color] = 'purple'                    
                weights[index_weight] = 5
            elif meeting_probability > r and man.is_contagious():
                friend.infect(viral_load)
                if friend.is_contagious():
                    colors[index_color] = 'purple'                                            
                    weights[index_weight] = 5
                else:
                    colors[index_color] = 'yellow'                    
                    weights[index_weight] = 5
            elif meeting_probability > r and friend.is_contagious():
                man.infect(viral_load_friend)
                if man.is_contagious():
                    colors[index_color] = 'purple'                                           
                    weights[index_weight] = 5
                else:
                    colors[index_color] = 'yellow'                                            
                    weights[index_weight] = 5
            # blue edges mean not contagious meeting happens
            elif meeting_probability > r and friend.is_contagious() == False and man.is_contagious() == False:
                colors[index_color] = 'blue'                                            
                weights[index_weight] = 1
            
            index_color += 1
            index_weight += 1
                    
        for man in patients:
            if man.is_contagious():
                day_contagious += 1
                node_color.append('red')
            else:
                node_color.append('green')

            man.sleep()
        
        filename = 'day' + str(day)
        
        # larger canvas
        plt.figure(3,figsize=(25,25)) 
        nx.draw_networkx(G, pos=nodes_pos, node_size=120, node_color=node_color, 
                         edge_color=colors, width=weights)        
        plt.title(filename, fontsize=40)
        plt.savefig(filename +'.png')
        plt.show()
        
        total_contagious.append(day_contagious)        
        day += 1
    print(G.number_of_edges())
    return total_contagious     

start_date = date(2020, 3, 1)
end_date = date(2020, 3, 31)
run_simulation(start_date, end_date, 0.8, 20)

